In [203]:
import pandas as pd
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv
from tqdm import tqdm
load_dotenv()

True

# Prompt Evaluation

In [ ]:
prompt_eval = """
Instruções:
Avalie a resposta gerada pela IA com base nos seguintes critérios:

1. Verifique se a Resposta da IA está contida na resposta Resposta do Especialista, ou seja, se não existe nenhuma contradição. Ignore termos diferentes ou pequenas informações a mais ou a menos.
2. A Resposta do Especialista pode conter mais informações do que foi solicitado na pergunta, se a informação da Resposta do Especialista não for necessária para responder a pergunta não use para avaliar a Resposta da IA.
3. Se a Resposta da IA tiver mais informações do que a Resposta do Especialista não deve ser levada em consideração para avaliação desde que as informações estejam corretas.
3. Verifique se a resposta pode responder à questão. Para isso veja se a resposta fornece as informações solicitadas na questão, se é suficiente. Por exemplo, se a questão pode ser respondida com um simples "Não", isso é aceitável.

Inclua um raciocínio que justifique a Avaliação. Se os critérios forem atendidos, retorne 'CORRETO'. Se qualquer um dos critérios não for atendido, retorne 'ERRADO'.

A Avaliação deve ser um objeto JSON, com as chaves resultado e raciocínio.

Exemplos:

1. 
### Questão:
Os rendimentos, decorrentes da prestação de serviços de consultoria técnica,  pagos por pessoa jurídica domiciliada no Brasil a sua matriz no exterior estão  sujeitos à legislação de preços de transferência?

### Resposta do Especialista:
Em primeiro lugar, há que distinguir se a prestação dos serviços no Brasil implicou transferência de tecnologia. Na hipótese de ficar comprovada a transferência de tecnologia, com a anuência do Instituto Nacional de Propriedade Industrial (INPI), a transação não estará sujeita às regras de preços de transferência consoante o estabelecido pelo art. 55 da IN RFB nº 1.312, de 2012. Nessa hipótese, a dedução de tais despesas está sujeita aos limites estabelecidos pelos arts. 362 a 365 do RIR/2018. Caso inexista transferência de tecnologia, esses serviços passam a se submeter às regras de preços de transferência.

### Resposta da IA:
Sim, estão sujeitos, a menos que haja transferência de tecnologia com anuência do INPI.

### Avaliação:
{{
  "raciocinio": "A resposta da IA está alinhada com a resposta do especialista, abordando corretamente a questão sem contradições, embora seja mais curta.",
  "resultado": "CORRETO"

}}

2. 
### Questão:
O que deve ser considerado como "contraprestações creditadas"?

### Resposta do Especialista:
Para efeito do disposto no art. 175 da Instrução Normativa RFB nº 1.700, de 2017, consideram-se contraprestações creditadas as contraprestações vencidas.

### Resposta da IA:
As contraprestações vencidas.

### Avaliação:
{{
  "raciocinio": "A resposta da IA cobre as principais hipóteses mencionadas pelo especialista, sem apresentar contradições, embora seja menos detalhada.",
  "resultado": "CORRETO"
}}

3. 
### Questão:
Existe prazo para a compensação de prejuízos fiscais da atividade rural?

### Resposta do Especialista:
Não existe qualquer prazo para compensação de prejuízos fiscais da atividade rural.

### Resposta da IA:
O prazo é de 7 dias a partir da data do prejuízo, podendo se extender até 30 dias para compensação de prejuízos fiscais da atividade rural.

### Avaliação:
{{
  "raciocinio": "A resposta da IA está incorreta porque menciona '7 dias' e 'até 30 dias', que contradizem a resposta do especialista.",
  "resultado": "ERRADO"
}}

4. 
### Questão:
A base de cálculo negativa da CSLL poderá ser compensada com resultados  apurados em períodos subsequentes?

### Resposta do Especialista:
Sim. A base de cálculo da CSLL, quando negativa, poderá ser compensada até o limite de 30% dos resultados apurados em períodos subsequentes, ajustados pelas adições e exclusões previstas na legislação.

### Resposta da IA:
Não. A base de cálculo da CSLL poderá ser compensada dos resultados apurados em períodos subsequentes.

### Avaliação:
{{
  "raciocinio": "A resposta da IA contradiz a resposta do especialista, fornecendo uma informação oposta sobre a necessidade possibilidade de compensação da CSLL.",
  "resultado": "ERRADO"
}}

Agora pense passo a passo e faça essa Avaliação:

### Questão:
{questao}

### Resposta do Especialista:
{resposta_especialista}

### Resposta da IA:
{resposta_ia}

### Avaliação:
"""


# Models Judgment

In [206]:
models = ['mistralai/Mixtral-8x7B-Instruct-v0.1',
               'mistralai/Mistral-7B-Instruct-v0.2',
               'zero-one-ai/Yi-34B-Chat',
               'garage-bAInd/Platypus2-70B-instruct',
               'google/gemma-7b-it',
               'lmsys/vicuna-13b-v1.5',
               'lmsys/vicuna-7b-v1.5',
               'meta-llama/Llama-2-70b-chat-hf',
               'meta-llama/Llama-2-13b-chat-hf',
               'meta-llama/Llama-2-7b-chat-hf',
               'openchat/openchat-3.5-1210',
               'WizardLM/WizardLM-13B-V1.2',
               'Qwen/Qwen1.5-14B-Chat',
               'Qwen/Qwen1.5-72B-Chat',
               'upstage/SOLAR-10.7B-Instruct-v1.0',
               'meta-llama/Llama-3-70b-chat-hf',
               'mistralai/Mistral-7B-Instruct-v0.3',
               'mistralai/Mixtral-8x22B-Instruct-v0.1',
               'Qwen/Qwen2-72B-Instruct',
               'meta-llama/Llama-3-8b-chat-hf',
               'Qwen/Qwen1.5-110B-Chat',
               'teknium/OpenHermes-2p5-Mistral-7B',
               'openia/gpt-3.5-turbo'
               ]

for m in tqdm(models, desc="Model Progress"):
    print(f"Processing model: {m}")
    df = pd.read_csv(f'result_question_answer_generation/{model.split('/')[1]}_QAG.csv') 
    prompt = ChatPromptTemplate.from_template(prompt_eval)
    model = ChatOpenAI(model='gpt-4o',temperature=0.1)
    output_parser = JsonOutputParser()
    total_questions = len(df)
    chain = prompt | model | output_parser
    data = []
    for i, l in enumerate(tqdm(df.itertuples(index=False), desc=f"Processing {m.split('/')[1]}", leave=False)):
        result = chain.invoke({'questao':l.Q,'resposta_especialista':l.A,'resposta_ia':l.A_model})
        data.append(result)
        if (i + 1) % 10 == 0 or (i + 1) == total_questions:
            print(f"Model {m}: Processed {i + 1}/{total_questions} questions")
    num_correto = 0
    num_errada = 0
    for r in data:
        if r['resultado'] == 'CORRETO':
            num_correto+=1
        else:
            num_errada+=1
            
    # Somar o total de entradas
    total_entradas = len(data)

    # Calcular acurácia
    acuracia = num_correto / total_entradas

    # Imprimir resultados
    print(f"Model: {m}")
    print(f"Correto: {num_correto}")
    print(f"Errada: {num_errada}")
    print(f"Total de Entradas: {total_entradas}")
    print(f"Acurácia: {acuracia:.2f}")
    df['Avaliacao_gpt4'] = data
    df.to_csv(f'result_question_answer_llm_evaluation/{model.split('/')[1]}_QAG_gpt-4o.csv', index=False)
    

Model Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Processing model: mistralai/Mixtral-8x7B-Instruct-v0.1


Model mistralai/Mixtral-8x7B-Instruct-v0.1: Processed 10/101 questions


Model mistralai/Mixtral-8x7B-Instruct-v0.1: Processed 20/101 questions


Model mistralai/Mixtral-8x7B-Instruct-v0.1: Processed 30/101 questions


...

Model teknium/OpenHermes-2p5-Mistral-7B: Processed 80/101 questions


Model teknium/OpenHermes-2p5-Mistral-7B: Processed 90/101 questions


Model teknium/OpenHermes-2p5-Mistral-7B: Processed 100/101 questions


Model Progress: 100%|██████████| 20/20 [1:13:01<00:00, 219.07s/it]

Model teknium/OpenHermes-2p5-Mistral-7B: Processed 101/101 questions
Model: teknium/OpenHermes-2p5-Mistral-7B
Correto: 56
Errada: 45
Total de Entradas: 101
Acurácia: 0.55
